## Dungeom: a de Bruijn graph-based genome assembler

### Introduction:
A de Bruijn graph can be used for the representation of a DNA sequence as the kmer components it is constituted from. If we define a kmer size, we can split a DNA sequence in its kmer components. After that, a directed graph can be constructed by connecting pairs of k-mers with overlaps between the first k-1 nucleotides and the last k-1 nucleotides. The direction of arrow goes from the k-mer, whose last k-1 nucleotides are overlapping, to the k-mer, whose first k-1 nucleotides are overlapping. This is known as de Brujin graph and provides an efficient way to assemble a DNA sequence.

### Description:
This de Bruijn graph-based assembler gets a DNA sequence and the number of kmer size as inputs and outputs the reassembled sequence and "identical" or "not identical" if the reassembled sequence is identical or not with the input sequence.

### Procedure:

1. The DNA sequence input is checked as to its length and the existance of non-standard nucleotides. The kmer size input is also checked as to whether it is a positive integer smaller than the length of the input sequence.
2. The sequence is broken into small fragments (kmers) and the unique kmers are saved into a set to secure their uniqueness (**nodes**). Also a list of tuples with the edges of the nodes is created that is later used in the construction of the de Bruijn graph (**edges**).
3. The de Bruijn graph is constructed and is visualized in the console.
4. All the edges are shuffled in order to secure the randomness of the assembly except the first one, in order to keep the starting point. The output will be the list of the shuffled edge tuples (**graph**).
5. The process of the assembly is based on finding the Eulerian path from the graph, that is the trail in the de Bruijn graph which visits every edge exactly once. That requires to keep track of the number of the edges for each node in a dictionary (**freq_dict**) and finish the assembly when all the edges are used (values in the dictionary are 0). The outcome of the procedure is a list (**tour_list**) with the nodes that were visited in the order that they were visited.
6. Finally, the nodes in the tour list are merged into the assembled sequence. Depending on whether the assembled sequence is identical to the input one, "Assembled sequence: identical" or "not identical" will also be printed.

First, let's load the necessary python modules.

In [ ]:
#import necessary packages
import sys
import argparse
from random import shuffle
print('Necessary packages loaded successfully!')

Next is the __input control__. In our case, our tool gets two inputs, a _DNA sequence_ and a _kmer size_. Using these three functions we can check:
* If the kmer size is a positive integer
* If the input DNA sequence is indeed a DNA sequence
* That the kmer size is not larger than the input DNA sequence

In [ ]:
def check_positive_kmer_size(value):
    if not value.isdigit():
        raise argparse.ArgumentTypeError("{} is an invalid positive integer value".format(value))
    ivalue = int(value)
    if ivalue <= 0:
         raise argparse.ArgumentTypeError("{} is an invalid positive integer value".format(value))
    return ivalue



def check_if_dna_sequence(input_seq):
    for nucleotide in input_seq:
        if nucleotide not in "GCTAgcta":
            raise argparse.ArgumentTypeError("Non-standard nucleotides in the sequence")
            
def check_if_kmer_largerthan_seq(value, input_seq):
    if int(value) >= len(input_seq):
        print("Dungeom.py: error: argument kmer: kmer size should be smaller than the sequence length")
        raise argparse.ArgumentTypeError("Kmer size should not exceed the length of the input sequence")

This is the cell where we give our input values.

In [ ]:
seq = "ATCGATCCCTGA"
kmer_size = "3"

Let's call the functions and control the input format and values.

In [ ]:
check_positive_kmer_size(kmer_size)
check_if_dna_sequence(seq)
check_if_kmer_largerthan_seq(kmer_size, seq)

print("Input is ok")

#corrects the maximum recursion error created by the find_eulerian_tour and find_tour functions if seq input is large
#if len(seq) > 38:
#    sys.setrecursionlimit(len(seq))

If everything is ok, we can proceed to the next function.  

The next function takes our _input DNA sequence_, breaks it into small fragments (kmers) of size equal to our _input kmer size_ and the unique kmers are saved into a python set to secure their uniqueness (**nodes**). Also a list of tuples with the edges of the nodes is created that is later used in the construction of the de Bruijn graph (**edges**).

In [ ]:
def de_bruijn_convert(sequence, kmer):
    kmer = int(kmer)
    edges = []
    nodes = set()
    for i in range(len(sequence) - kmer):
        edges.append((sequence[i:i+kmer], sequence[i+1:i+kmer+1]))
        nodes.add(sequence[i:i+kmer])
        nodes.add(sequence[i+1:i+kmer+1])
    return edges, nodes

In [ ]:
edges, nodes = de_bruijn_convert(seq , kmer_size )
print("nodes:")
print(nodes)
print("edges:")
print(edges)

In the next cell, we are going to construct and vizualize the generated de Bruijn graph using the **_graphviz_** graph visualization software. Nevertheless, it needs to be installed in your system by:
- first installing the source code of the software (https://graphviz.gitlab.io/download/)
- then installing the Python package (_pip install graphviz_)

**Note**: This step is not essential for downtream analysis, however it provides a nice illustration of how a de Brujin graph of our input sequence will look like.

Example: Given the input sequence "ATCGATCCCTGA" and kmer size = 3, we get:
<img src="example_graphviz.jpg">  
**Second note**:  In the above directed graph, assuming that we visit each edge once and we start from the _ATC_ kmer, we can see that there is only one path (solution), so the sequence will always be assembled in the correct way. However this is not always the case (e.g. one node can be connected to more than 1 nodes having the same edge number in both "parallel" paths). This could occur if we set a different (smaller) kmer size.

In [ ]:
from graphviz import Digraph

f = Digraph('de_Bruijn_graph', filename = "dgb.jpg")
f.attr(rankdir='LR', size='15')
for node in nodes:
    f.node(node)
for lf, rg in edges:   
    f.edge(lf, rg)
f    

Next we will use a function that will shuffle all the edge tuples in order to secure the randomness of the assembly except the first one, in order to keep the starting point. The output will be the list of the shuffled edge tuples (**graph**).

In [ ]:
from random import shuffle
def shuffled_edges(graph):
    first_element = True
    for edge in graph:
        if first_element:
            first_edge = edge
            graph.remove(edge)
            first_element = False      
    shuffle(graph)
    graph.insert(0, first_edge)
    return graph

In [ ]:
graph = shuffled_edges(edges)
print("graph:")
print(graph)

The process of the assembly is based on finding the _Eulerian path_ from the graph, that is the trail in the de Bruijn graph which visits each edge exactly once. That requires to keep track of the number of the edges for each node in a dictionary (**freq_dict**) using the next function.

In [ ]:
def dictionary_of_node_edge_frequencies(edges, nodes):
    nodes = list(nodes)
    result = [0 for i in range(len(nodes))]
    for node in nodes:
        for (a,b) in edges:
            if node == a or node == b:
                result[nodes.index(node)] += 1
    edge_freq = dict(zip(nodes, result))
    return edge_freq

In [ ]:
freq_dict = dictionary_of_node_edge_frequencies(graph, nodes)
print("freq_dict:")
print(freq_dict)

Now it's time to take the _Eulerian path_: The assembly will be finished when all the edges have been visited (values in the **freq_dict** dictionary become 0). The first function (find_eulerian_tour) makes use of the second function (find_tour) to  visit each node depending on their edges recursively. The outcome of the procedure is a list (**tour_list**) with the nodes that were visited in the order that they were visited.

In [ ]:
def find_eulerian_tour(graph):
    edges, nodes = de_bruijn_convert(seq, kmer_size)
    freq_dict = dictionary_of_node_edge_frequencies(graph, nodes)
    while sum(freq_dict.values()) != 0:
        edges, nodes = de_bruijn_convert(seq, kmer_size)
        graph = shuffled_edges(edges)
        freq_dict = dictionary_of_node_edge_frequencies(graph, nodes)
        tour_list = []
        find_tour(graph[0][0],graph,tour_list,freq_dict)
    return tour_list

def find_tour(current,Edges,tour_list,freq_dict):
    for (a,b) in Edges:
        if a==current:
            Edges.remove((a,b))
            freq_dict[a] -= 1
            freq_dict[b] -= 1
            find_tour(b,Edges,tour_list,freq_dict)
    tour_list.insert(0,current)

In [ ]:
tour_list = find_eulerian_tour(graph)
print("tour_list:")
print(tour_list)


Finally, the nodes in the **tour_list** are merged into the assembled sequence. Depending on whether the assembled sequence is identical to the input one, "Assembled sequence: identical" or "not identical" will also be printed.

In [ ]:
def eulerian_tour_to_superstring(tour_list):
    superstring = ""
    first_word = True
    for word in tour_list:
        if first_word:
            superstring += word
            first_word = False
        else:
            superstring += word[-1]
    return superstring

In [ ]:
assembled_seq = eulerian_tour_to_superstring(tour_list)
print("Assembled sequence: " + assembled_seq)
if assembled_seq == seq:
    print("Identical to input sequence")
else:
    print("Not identical to input sequence")